In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal
from collections import OrderedDict
from matplotlib import rcParams

In [ ]:
rcParams.update({'legend.fontsize': 'xx-large',
                 'axes.labelsize': 'xx-large',
                 'axes.titlesize':'xx-large',
                 'xtick.labelsize':'xx-large',
                 'ytick.labelsize':'xx-large'})

# P3 - Deterministic Optimization

In [ ]:
coefficients_table = np.asarray([
    [-0.195 , 0.95],
    [-0.975 , 0.95],
    [-1.5955, 0.95],
    [-1.9114, 0.95]
])
pd.DataFrame(coefficients_table, index='exp_0 exp_1 exp_2 exp_3'.split(), columns='c_0 c_1'.split())

## a)

Implement the subsystems 'source'.

In [ ]:
def apply_source_model(u, coefficients):
    raise NotImplementedError

Test your implementation of the 'source':

In [ ]:
test_signal = np.asarray([1., 2., 3., 4., 5.])
test_coefficients = np.asarray([3., -4.])
desired = np.asarray([1., -1., 10., -30., 135.])
actual = apply_source_model(test_signal, test_coefficients)
np.testing.assert_equal(actual, desired)

Implement the subsystems 'predictor'.

In [ ]:
def apply_predictor_model(x, coefficients):
    raise NotImplementedError

Test your implementation of the 'predictor':

In [ ]:
test_signal = np.asarray([1., 2., 3., 4., 5.])
test_coefficients = np.asarray([1., 1.])
desired = np.asarray([1., 3., 5., 7., 9.])
actual = apply_predictor_model(test_signal, test_coefficients)
np.testing.assert_equal(actual, desired)

## b)

Compute the variance $\sigma_u^2$ as a function of $\sigma_x^2$ and the source filter coefficients $a_1$ and $a_2$.

In [ ]:
def get_sigma_u(sigma_x, coefficients):
    raise NotImplementedError

## c)

Perform the iterative gradient descent (Hint: To keep Python from ending up in an endless loop, always use a second stopping-criterion to exit the loop after e.g. $10000$ iterations.). Calculate the Eigenvalues, the Eigenvalue-Spread, the optimal stepsize $\mu_{opt}$ and the number of steps needed till the iteration stops for the different pairs of $a_1$ and $a_2$.

In [ ]:
def get_auto_correlation_matrix(sigma_x, coefficients):
    raise NotImplementedError

In [ ]:
def get_cross_correlation_vector(sigma_x, coefficients):
    raise NotImplementedError

In [ ]:
def get_cost(sigma_x, w, p, R):
    raise NotImplementedError

In [ ]:
mu = 'mu_opt'
N = 100000
K = 2
sigma_x = 1.
max_iterations = 10000000
eps = 1e-4
summary = []
for experiment_index in range(coefficients_table.shape[0]):
    source_coefficients = coefficients_table[experiment_index, :]
    sigma_u = get_sigma_u(sigma_x, source_coefficients)
    u = np.random.normal(scale=sigma_u, size=(N,))
    x = apply_source_model(u, source_coefficients)
    
    R = get_auto_correlation_matrix(sigma_x, source_coefficients)
    p = get_cross_correlation_vector(sigma_x, source_coefficients)
    
    eigenvalues, eigenvectors = ???
    eigenvalue_spread = ???
    if mu == 'mu_opt':
        mu = ???
    
    J_history = list()
    v_history = list()
    w_history = list()
    
    w = np.zeros((K, 1))
    w_opt = ???
    w_history.append(w)
    iteration = 0
    while ???:
        J_history.append(???)
        v_history.append(???)
        w = ???
        w_history.append(w)
        iteration += 1
    y = apply_predictor_model(x, w.flatten())
    J_actual = ???
    
    summary.append(OrderedDict(
        c_0=source_coefficients[0],
        c_1=source_coefficients[1],
        w_0=w[0, 0],
        w_1=w[1, 0],
        w_opt=w_opt,
        w_history=np.asarray(w_history),
        v_history=np.asarray(v_history),
        J_history=np.asarray(J_history),
        lambda_0=eigenvalues[0],
        lambda_1=eigenvalues[1],
        mu=mu,
        spread=eigenvalue_spread,
        J_theoretical=np.round(J_history[-1], 6),
        J_actual=np.round(J_actual, 6),
        steps=iteration,
        p=p,
        R=R
    ))

df = pd.DataFrame(summary)
df.reindex(columns='c_0 c_1 w_0 w_1 lambda_0 lambda_1 mu spread J_theoretical J_actual steps'.split())

# d)

Display the development of $\mathbf w(\kappa)$.

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))
for experiment, axis in zip(summary, axes.flatten()):
    delta = 0.05
    x = np.arange(-3, 3, delta)
    y = np.arange(-3, 3, delta)
    X, Y = np.meshgrid(x, y)
    Z = np.empty_like(X)
    for ix in range(X.shape[0]):
        for iy in range(X.shape[1]):
            Z[ix, iy] = get_cost(sigma_x, np.asarray([[X[ix, iy]], [Y[ix, iy]]]), experiment['p'], experiment['R'])
    axis.plot(experiment['w_history'][:, 0], experiment['w_history'][:, 1])
    CS = axis.contour(X, Y, Z, np.arange(5))
    plt.clabel(CS, inline=1, fontsize=10)
    axis.set_title(f'c_0 = {experiment["c_0"]}, c_1 = {experiment["c_1"]}', fontsize=20)
    axis.set_xlabel(r'$w_0(\kappa)$')
    axis.set_ylabel(r'$w_1(\kappa)$')
    axis.set_xticks(np.linspace(-3, 3, 7, endpoint=True))
    axis.set_yticks(np.linspace(-3, 3, 7, endpoint=True))
plt.tight_layout()

Display the development of $\boldsymbol \nu(\kappa)$.

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))
for experiment, axis in zip(summary, axes.flatten()):
    delta = 0.05
    x = np.arange(-3, 3, delta)
    y = np.arange(-3, 3, delta)
    X, Y = np.meshgrid(x, y)
    Z = np.empty_like(X)
    for ix in range(X.shape[0]):
        for iy in range(X.shape[1]):
            Z[ix, iy] = get_cost(sigma_x, eigenvectors.T @ np.asarray([[X[ix, iy]], [Y[ix, iy]]]) + experiment['w_opt'], experiment['p'], experiment['R'])
    axis.plot(experiment['v_history'][:, 0], experiment['v_history'][:, 1])
    CS = axis.contour(X, Y, Z, np.arange(5))
    plt.clabel(CS, inline=1, fontsize=10)
    axis.set_title(f'c_0 = {experiment["c_0"]}, c_1 = {experiment["c_1"]}', fontsize=20)
    axis.set_xlabel(r'$\nu_0(\kappa)$')
    axis.set_ylabel(r'$\nu_1(\kappa)$')
    axis.set_xticks(np.linspace(-3, 3, 7, endpoint=True))
    axis.set_yticks(np.linspace(-3, 3, 7, endpoint=True))
plt.tight_layout()

# e)
Display the development of the cost function $J(\kappa)$.

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))
for experiment, axis in zip(summary, axes.flatten()):
    axis.plot(experiment['J_history'])
    axis.set_title(f'c_0 = {experiment["c_0"]}, c_1 = {experiment["c_1"]}', fontsize=20)
    axis.set_xlabel(r'$\kappa$')
    axis.set_ylabel(r'$J(\kappa)$')
plt.tight_layout()

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))
for experiment, axis in zip(summary, axes.flatten()):
    axis.plot(experiment['w_history'][:, 0])
    axis.plot(experiment['w_history'][:, 1])
    axis.set_title(f'c_0 = {experiment["c_0"]}, c_1 = {experiment["c_1"]}', fontsize=20)
    axis.set_xlabel(r'$\kappa$')
    axis.set_ylabel(r'$w(\kappa)$')
    axis.legend(r'$w(\kappa)=w_0(\kappa)$ $w(\kappa)=w_1(\kappa)$'.split())
plt.tight_layout()